## Language Modeling Overview

- Text Classification은 Continious Bag of N-gram(Words)를 사용하면, 대부분 잘 할 수 있다고 언급했었으며 이는 Sentence Representation이나 Variable-length sequence에 대한 fixed-length representation을 찾고 생각해보는 문제 였다.

- 이제 다루게 될 NLP의 큰 파트인 Language Generation을 위해서는 Language Model에 대한 이해가 필요
- 사실 Language Model은 Generate를 하지는 않지만, Language Model은 Language Generation과 연관이 있는 문제이다.

- Language Modeling
    - Input : a Sentence
    - Output : the probability of the input sentence
    - 문장이 주어졌을 때, 이 문장의 확률을 알려주는 것 (얼마나 likely 한지)
        - cf) HMM의 Evaluation Question과 비슷한 것 같다? (기계개론2 HMM)
        
    - 문장에 대해 Scoring 하는 것으로 Unsupervised Learning에 속하지만, sentenc scoring과 generation은 equivalent한 문제임을 알 수 있다.
    -  Sentence에 대해서 Scoring을 굉장히 잘하는 Language Model이 있다면 이 모델을 바꾸어 Generation 문제에서 모든 가능한 Sentence에 대해서 Scoring을 해 제일 좋은 애를 뽑는 문제로 생각할 수 있음. 그리고 Prefix를 보고, 또는 Pruning과 같은 방법으로 효율적으로 search 할 수 있을 것이다.
    - 굉장히 다른 문제 같아 보여도 NN에서는 이를 합쳐 자연스럽게 할 수 있다.


### 1)  Autoregressive language modeling

- 어떤 Sequence가 주어졌을 때, 어떻게 Probability Distribution을 계산할 수 있을까?
- Sequence에 어떻게 Probability(확률)를 부여(Scoring)할 수 있을까?

- 그 방법은 Sequence의 확률은 이전 단어들이 주어졌을 때, 다음 단어가 나올 확률(distribution이 어떤지)들을 전부 곱한 것으로 생각해보자.

- Autoregressive sequence modelling
    - The distribution over the next token is based on all the previous tokens.
    - $ P(X) = p(x_{1})p(x_{2}|x_{1})\cdots p(x_{T}|x_{1},...,x_{T-1}) $
    - This equality holds exactly due to the def. of conditional distribution
    - $ \to $ 이 방법의 좋은 이유: 이렇게 정의 하면서 비지도학습 문제를 지도학습으로 푸는 문제로 변함.
        - input output, pair에 대해 Learning하므로 (input 은 이전에 나온 토큰, output은 다음에 나올 토큰을 예측 )
    - 이렇게 생각하면, Multi-Class (Text) Classification 문제와 같아지면서 이전에 배운 모델들을 고스란히 사용할 수 있다.
        - language modeling, sentence Scoring은 Text Classification을 많이 하는 것.

- Each conditional is a sentence classifier
- Autoregressive sequence modelling: $ p(X) = \prod_{t=1}^{T} p(x_{t}|x_{<t}) $
- Loss function: the sum of negative log-probabilities (NLL Loss) [NLL Loss 설명](https://jaejunyoo.blogspot.com/2018/02/minimizing-negative-log-likelihood-in-kor.html?fbclid=IwAR3emBTGpp9vsRMh_ZzQbw7kU0WmNXadw6zqBCp0xpDJUAC4xHJrbzH1b28) <br>
    - $ log p_{\theta}(X) \sum_{n=1}^{N}\sum_{t=1}^{T}logp_{\theta}(x_{t}|x_{<t}) $
<img  src="./image/img_AutoregressiveSM.PNG" width="70%">        

- 이렇게 생각하는 것 자체가 자연스럽다. Speaking Language에서, 인간이 말하는 사고방식에서와 비슷하다. (Written은 좀 다르지만)

- Scoring a sentence
    - example sentence: "In Korea, more than half of residents speak Korean."
        - "In" is a reasonable token to start a sentence.
        - “Korea” is pretty likely given “In”
        - &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $ \vdots $
        - “half ” is also very likely after “In Korea, more than”
    - 최종적으로
    - "In Korea, more than half of residents speak" 이후에 Finnish 보다, Korean의 score가 높게 나와 Korean의 단어를 선택하도록 하는 것.

### Before "DL World"

- NN이 이렇게 많이 쓰이기 이전에는 어떻게 이 문제를 해결했는지 보고, 왜 Neural Net을 쓰고 싶어했는지를 알아보자.

- 80년대, IBM에서 통계에 기반한 음성 인식을 시작하고, 통계적 기계 번역을 시도했을 때, IBM은 Language Modeling에 대한 필요성을 느낀다. IBM은 음성인식 시스템을 잘 만드는 것이 힘드니까 Rough하게 음성인식하는 머신이 여러 가능성있는  문장들을 주면 그 중에서 Language Model을 이용해 가장 말이되는(score가 높은) 문장을 뽑아내겠다는 시도를 했다.(Machine Translation도 동일하게)

- 당시의 Computing의 환경이 열악했으므로 MLE를 사용했었다.

- We need to estimate n-gram probabilities: $ p(x|x_{-N},x_{-N+1}...,x_{-1}) $
    - 이것에 대해 conditional and marginal probabilities.를 활용해
    - $ p(x|x_{-N},x_{-N+1}...,x_{-1}) = {{p(x,x_{-N},x_{-N+1}...,x_{-1})} \over {p(x_{-N},x_{-N+1}...,x_{-1})}} = {{p(x,x_{-N},x_{-N+1}...,x_{-1})} \over {\sum_{x \in V} p(x,x_{-N},x_{-N+1}...,x_{-1})}} $
        - V: all possible tokens(Vocabulary)

- How do we estimate the probability?
    - MLE: 가장 간단한 estimation method이면서 가장 많이 사용.
    - 동전던지기에서 MLE로 구하는 앞면 확률은 결국(극점을 이용한다고 배움) 앞면/횟수이니까 Large Scale Data Science에서 많이 쓰이는 것이 counting만 잘하면 되니까. 
    - MLE에서 toss the coin a lot and look at how often it lands heads. 
        - $ \to $ n-gram에서  Data Collection(toss the coin a lot) , Estimation (look at how often it lands heads) 이 된다.
    - 이러면 2가지를 Estimate하는 것(계산하는 것)
        -  주어진 n-gram에 대해 몇 번 나왔는지 Count the # of occurrences for the n-gram: $ (x_{-N},x_{-N+1}...,x_{-1},x) $

        - 주어진 n-gram에서 앞에 N-1개 빼고, 나머지가 다 바뀐다고 했을 때 몇 번이 나왔는지. Count the #’s of occurrences for all the n-grams of the form: $ (x_{-N},x_{-N+1}...,x_{-1},?) $

    - 그러면 위위 꺼에 위꺼로 나누면 MLE가 나와 N-gram prob.를 계산할 수 있다. 이를 계산해 놓으면 새로운 문장이 주어질 때, 앞에서 부터 쭉가면서 확률을 계산하여 MLE of sentence prob이 나올 것이다. 아래처럼
 
- We need to estimate n-gram probabilities:
    - $ p(x|x_{-N},x_{-N+1}...,x_{-1}) = {{p(x,x_{-N},x_{-N+1}...,x_{-1})} \over {p(x_{-N},x_{-N+1}...,x_{-1})}} $

- Estimation:
    - $ \begin{matrix} p(x|x_{-N},x_{-N+1}...,x_{-1}) &=& {{p(x,x_{-N},x_{-N+1}...,x_{-1})} \over {\sum_{x \in V} p(x,x_{-N},x_{-N+1}...,x_{-1})}} \\ &\approx& {{p(x,x_{-N},x_{-N+1}...,x_{-1})} \over {\sum_{x' \in V} p(x',x_{-N},x_{-N+1}...,x_{-1})}} \end{matrix} $ 
    - 이 과정은 당연하다.

- Example: How likely is “University” given “New York”?
    - Count all "New York University", 데이터 안에서 다 세본다.
    - trigram을 세본다. "New York ?": e.g., "New York State", "New York City", "New York Police", ...
    - 이중에 "New York University"는 몇 번이었나.
    
- But 이 방식의 2 Big problem!  
    - Data sparsity: lack of generalization
        - ex) "a lion is chasing a llama"라는 문장이 굉장히 likely 임에도 불고하고 데이터에는 없을 때. 확률이 0으로 전체 확률이 0이 되는
    - Inability to capture long-term dependencies
        - long-term dependencies를 어떻게 극복할 것인가. Sparsity vs N-gram
    - 하지만, long-term dependencies를 language model에서 capture하는 것이 의미가 있는지 또한 큰 debate point이다.
        - cf) 영어는 n=7이면 거의 완벽하게 잘된다. (SVO구조 때문?)
        - 하지만 common을 놓치는 것과 corner case를 놓치지 않는 것도 중요하다.
        - ex) Consider "the same ** stump ** which had impaled the car of many a guest in the past thirty years and which <u> he refused to have </u> ** removed **"
        - removed의 prob를 계산한다고 할 때, he refused to have로 예측하는 것은 거의 불가능하다. stump까지가야 바로 알 수 있는 이런 issue.

- Traditional solutions for 2 Big problem! 
    -  Solutions 4 Data sparsity:
        - a.  smoothing technique: 근데 한번도 안나온 것도 다 똑같이 count하는게 말이 되나?
        - b. Backoff: count를 볼 때 없으면 0을 쓰는 것이아니라 (n-1)gram을 본다.
            - ex) "chasing a llama"가 한 번도 안나오면 a llama를 대신 보고, 확률에 맞게 조정해준다.
        - c. Kneser-Ney smoothing/backoff(The most widely used approach): 가장 많이 쓰이는 테크닉으로 이것과 N-gram섞어 쓰면 굉장히 좋고, 제대로 직접 count하면 성능이 많이 안좋아 질 수 있으므로 라이브러리를 사용하자. 최신의 것을 사용하면 좋다.
            -  KenLM 패키지
        
    - Solutions 4 long-term dependencies:
        - count based n-gram에서는 거의 해결할 수 없다.
        - dependency parsing으로 해결 문장 앞 부분과 뒷부분을 잇는 그래프를 만들어 grammitical하게. 
            - Consider "the same ** stump ** which had impaled the car of many a guest in the past thirty years and which <u> he refused to have </u> ** removed **"
        - 여기서 stump와 removed 는 바로 edge가 하나 연결되어 reorder 할 수 있게 된다.
        - 사실 이것도 어렵다는 듯하다.
        

### Reference: 

- https://www.edwith.org/deepnlp 조경현교수님, 딥러닝을 이용한 자연어 처리 강의 및 강의 자료 

NLL Loss 설명
- https://jaejunyoo.blogspot.com/2018/02/minimizing-negative-log-likelihood-in-kor.html?fbclid=IwAR3emBTGpp9vsRMh_ZzQbw7kU0WmNXadw6zqBCp0xpDJUAC4xHJrbzH1b28